In [1]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import requests
from selenium import webdriver

In [2]:
#Use Beautiful Soup to pull all contents of the web-page
page = requests.get('https://en.wikipedia.org/wiki/UEFA_Cup_and_Europa_League_records_and_statistics')
soup = BeautifulSoup(page.content,'html.parser')

#Find all tables in the page
tables = soup.findAll("table")

In [3]:
table_header = tables[0].findAll("tr")
header_list=[]

In [13]:
for i in range(1,len(table_header[0])):
    if(i==1):
        header_list.append(table_header[0].contents[i].contents[0].lstrip().rstrip())
    elif (i%2 != 0):
        header_list.append(table_header[0].contents[i].contents[0].lstrip().rstrip())

In [77]:
header_list

['Club',
 'Finals Appearances',
 'Winners',
 'Runners-up',
 'Years won',
 'Years runners-up']

In [78]:
del header_list[1]

In [79]:
header_list

['Club', 'Winners', 'Runners-up', 'Years won', 'Years runners-up']

In [81]:
el_final_record = pd.DataFrame(columns=header_list)
rows = tables[0].findAll("tr")

In [82]:
rows[1].contents[1].contents[2].contents[0]
rows[1].contents[5].contents[0].rstrip()
rows[1].contents[7].contents[0].rstrip()
x = rows[1].contents[9].findAll("a")

In [90]:
for club in range(1,len(rows)):
    club_name = rows[club].contents[1].contents[2].contents[0]
    titles = rows[club].contents[5].contents[0].rstrip()
    runners = rows[club].contents[7].contents[0].rstrip()
    x = rows[club].contents[9].findAll("a")
    win_list = []
    for i in range(0,len(x)):
        win_list.append(x[i].contents[0])
    y = rows[club].contents[11].findAll("a")
    runner_list = []
    for j in range(0,len(y)):
        runner_list.append(y[j].contents[0])
    el_final_record.loc[club-1] = [club_name,titles,runners,win_list,runner_list]

In [100]:
semi_table_hdr = tables[6].findAll("tr")
semi_hdr_list = []

In [103]:
for i in range(1,len(semi_table_hdr[0])):
    if (i%2 != 0):
        semi_hdr_list.append(semi_table_hdr[0].contents[i].contents[0].lstrip().rstrip())

In [111]:
el_semi_record = pd.DataFrame(columns=semi_hdr_list)
rows = tables[6].findAll("tr")

In [114]:
club_name = rows[1].contents[1].contents[2].contents[0]

In [118]:
for club in range(1,len(rows)):
    club_name = rows[club].contents[1].contents[2].contents[0]
    number_of_semis = rows[club].contents[3].contents[0].rstrip()
    x = rows[club].contents[5].findAll("a")
    years = []
    for i in range(0,len(x)):
        years.append(x[i].contents[0])
    el_semi_record.loc[club-1] = [club_name,number_of_semis,years]

In [116]:
el_semi_record

,Team,No.,Years
0,Internazionale,7,"[1985, 1986, 1991, 1994, 1997, 1998, 2002]"
1,Juventus,6,"[1975, 1977, 1990, 1993, 1995, 2014]"
2,Borussia Mönchengladbach,5,"[1973, 1975, 1979, 1980, 1987]"
3,Liverpool,5,"[1973, 1976, 2001, 2010, 2016]"
4,Atlético Madrid,5,"[1998, 1999, 2010, 2012, 2018]"
5,Sevilla,5,"[2006, 2007, 2014, 2015, 2016]"
6,Tottenham Hotspur,4,"[1972, 1973, 1974, 1984]"
7,1. FC Köln,4,"[1975, 1981, 1986, 1990]"
8,Barcelona,4,"[1976, 1978, 1996, 2001]"
9,Hamburger SV,4,"[1976, 1982, 2009, 2010]"
